In [35]:
import pandas as pd
import spacy
from lemminflect import getAllInflections
import random
import pattern.en

nlp = spacy.load("en_core_web_sm")

In [3]:
japonaisCorpus = pd.read_csv("../ja/japonaisExercices.tsv", sep="\t", encoding="utf-8").dropna()
japonaisCorpus

,sourceWord,targetWord,minSource,maxSource,minTarget,maxTarget
1,because of,のため,tom died because of me.,she was born just a generation past slavery; a...,嵐で停電した。,彼女は奴隷制終了のちょうど一世代後に生まれました。時は道に車もなく、そらに飛行機もなく時代で...
2,be out of,から出る,the lock must be out of order.,because the japanese morning train is very cro...,数日町を離れます。,日本の朝の電車はとても混んでいるので、朝の８時に急行電車に乗ろうものなら、息が出来ませんよ。
3,on an empty stomach,空腹時に,drinking on an empty stomach is bad for your h...,drinking on an empty stomach is bad for your h...,空きっ腹にお酒を飲むのは体に良くない。,空きっ腹にお酒を飲むのは体に良くない。
4,?,？,i wonder if i should tell him … ?,is it ok if i ask a stupid question? can you r...,彼に話すのかな。,「おバカチックな質問していい？これなんて読むの？コロナ・・・？うず？なべ？違うよね」「コロナ...
7,on the contrary,それどころか,"i thought he was busy, but on the contrary he ...","i thought he was busy, but on the contrary he ...",彼は忙しいと思ったが、それどころか暇だった。,彼は忙しいと思ったが、それどころか暇だった。
...,...,...,...,...,...,...
2921,maiden name,旧姓,mary's maiden name is jackson.,mary's maiden name is jackson.,メアリーの旧姓は、ジャクソンだよ。,メアリーの旧姓は、ジャクソンだよ。
2926,tell the truth,実を言うと,he would not tell the truth after all.,he didn't tell the truth for fear she should g...,彼はついに真実を明かそうとはしなかった。,彼は彼女が怒らないように本当のことを言わなかった。
2929,cap,キャップ,which cap is yours?,"remove the cap from the ink refill bottle, fil...",君の帽子は、どれ？,インキ補充瓶のキャップを外し、スポイトでインキを吸い上げ、マーカーの吸収体へ適量、滴下してく...
2930,several,several,several years went by.,although most of the content and thought has n...,電話が数度鳴った。,ドイツでは今日、いくつかの都市で襲撃に反対する抗議行動が催されましたが、その都市の中には、月...


In [4]:
# Test without conjugator using verb inflections

for sentence in japonaisCorpus["minSource"][:10]:
    doc = nlp(sentence)
    for i, token in enumerate(doc):
        if token.pos_ == "VERB":
            inflections = [inflection[0] for inflection in getAllInflections(token.lemma_, upos="VERB").values()]
            s = sentence.replace(token.text, random.choice(inflections))
    print([sentence, s])

['tom died because of me.', 'tom dies because of me.']
['the lock must be out of order.', 'tom dies because of me.']
['drinking on an empty stomach is bad for your health.', 'drink on an empty stomach is bad for your health.']
['i wonder if i should tell him … ?', 'i wonder if i should tell him … ?']
['i thought he was busy, but on the contrary he was idle.', 'i thought he was busy, but on the contrary he was idle.']
["let's go out unless it rains.", "let's go out unless it rained."]
['come again tomorrow.', 'come again tomorrow.']
['tom is at the end of the line.', 'come again tomorrow.']
['all will go except you.', 'all will go except you.']
["it's starting now.", "it's start now."]


In [258]:
# Test using conjugator from pattern

for sentence in japonaisCorpus["minSource"][:10]:
    doc = nlp(sentence)
    s = ""
    for i, token in enumerate(doc):
        if token.pos_ == "VERB":
            tense, person, number, mood, aspect = random.choice(pattern.en.tenses(token.lemma_))
            newVerb = pattern.en.conjugate(token.lemma_, tense=tense, person=person, number=number, mood=mood, aspect=aspect)
            s = sentence.replace(token.text, newVerb)
    print([sentence, s])

['tom died because of me.', 'tom die because of me.']
['the lock must be out of order.', '']
['drinking on an empty stomach is bad for your health.', 'drink on an empty stomach is bad for your health.']
['i wonder if i should tell him … ?', 'i wonder if i should tell him … ?']
['i thought he was busy, but on the contrary he was idle.', 'i think he was busy, but on the contrary he was idle.']
["let's go out unless it rains.", "let's go out unless it rain."]
['come again tomorrow.', 'come again tomorrow.']
['tom is at the end of the line.', '']
['all will go except you.', 'all will go except you.']
["it's starting now.", "it's start now."]
